# Fuction Calling

<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/function_calling.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

A function calling occurs when you interact with the Chat API to communicate with a Language Learning Model (LLM). Within the tool array, you have the flexibility to define custom functions. This capability enables the model to dynamically generate and provide function signatures in JSON format, facilitating seamless integration with external tools and applications.

With function calling, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The model does not call the function, but it generates JSON that you can use to call the function in your code.

1. Make assistant use external api.
2. Call api with natural language.

In [1]:
#@title Install requirements
!pip install -qU openai
!pip install -qU python-dotenv

In [22]:
#@title set API key
from pprint import pprint
import os

import warnings
warnings.filterwarnings('ignore')

from IPython import get_ipython

upstage_api_key_env_name = 'UPSTAGE_API_KEY'
def load_env():
    if 'google.colab' in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata
        upstage_api_key = userdata.get(upstage_api_key_env_name)
        return os.environ.setdefault('UPSTAGE_API_KEY', upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv
        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)

UPSTAGE_API_KEY = load_env()

In this example, we'll use function calling in the following order.

1. Make an API call with tools parameter, which has function signature and description.
2. API response contains tool calls
3. Use it to call our function
4. Make another API call with the response from our function appended to our original message list

In [8]:
#@title Define API
#@markdown In this notebook, We'll define a dummy function that replaces the actual external API.

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "seoul" in location.lower():
        return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [21]:
#@title Request conversation and available functions to the model

from openai import OpenAI
import json

client = OpenAI(api_key=UPSTAGE_API_KEY, base_url="https://api.upstage.ai/v1/solar")

messages = [
    {
        "role": "user",
        "content": "What's the weather like in San Francisco, Seoul, and Paris?",
    }
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# Request conversation
response = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

print("Response message:\n", response_message.tool_calls)

Response message:
 [ChatCompletionMessageToolCall(id='e3f3546e-523d-4e94-900b-3bee355ed0fb', function=Function(arguments='{"location":"San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='15fc9fb5-a969-45f6-af76-966813ceee4f', function=Function(arguments='{"location":"Seoul"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='7e055ec9-3a02-4c95-8604-b0d2fa7c8b78', function=Function(arguments='{"location":"Paris"}', name='get_current_weather'), type='function')]


In [22]:
#@title Check if the model wanted to call a function and call the function
if tool_calls:
    available_function = {
        "get_current_weather": get_current_weather
    }
    messages.append(response_message)

    # Send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_function[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(location=function_args.get("location"), unit=function_args.get("unit"))

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )
    print("messages:\n", messages, "\n")
    
    second_response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=messages
    )
    
    print("second_response.choices[0].message.content)

messages:
 [{'role': 'user', 'content': "What's the weather like in San Francisco, Seoul, and Paris?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='119a29d5-ac04-4985-963e-4944060242e9', function=Function(arguments='{"location":"San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='3e319c52-1cec-42c0-9fb0-997b10780eaa', function=Function(arguments='{"location":"Seoul"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='d6ea1dbc-467b-4704-a0b2-c9f952c4fb5f', function=Function(arguments='{"location":"Paris"}', name='get_current_weather'), type='function')]), {'tool_call_id': '119a29d5-ac04-4985-963e-4944060242e9', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "San Francisco", "temperature": "72", "unit": null}'}, {'tool_call_id': '3e319c52-1cec-42c0-9fb0-997b10780eaa', 'role': 'tool', 'name': 'get_current_weathe